Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# Библиотеки и т.п.

Установка библиотек

In [ ]:
%tensorflow_version 1.x
!pip uninstall stable-baselines -y
!pip install stable-baselines gym-retro

TensorFlow 1.x selected.
Uninstalling stable-baselines-2.2.1:
  Successfully uninstalled stable-baselines-2.2.1
     |████████████████████████████████| 245kB 14.8MB/s 
     |████████████████████████████████| 162.0MB 91kB/s 


Rom (образ)

In [ ]:
!python -m retro.import /gdrive/My\ Drive/Retro/mkii/rom/

Importing MortalKombatII-Genesis
Imported 1 games


Подключение библиотек

In [ ]:
import gym
import retro
from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy, MlpLnLstmPolicy, CnnLnLstmPolicy, CnnPolicy, CnnLstmPolicy
from stable_baselines.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines import PPO2, A2C
import numpy as np
from stable_baselines.common.callbacks import CheckpointCallback

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Среда

Управление

In [ ]:
class Discretizer(gym.ActionWrapper):
    def __init__(self, env):
        super(Discretizer, self).__init__(env)
        buttons = ["B", "A", "MODE", "START", "UP", "DOWN", "LEFT", "RIGHT", "C", "Y", "X", "Z"]
        actions = [[], ['X'], ['A'], ['Z'], ['C'], ['Y'], ['START'], ['UP'], ['DOWN'], ['LEFT'], ['RIGHT'], 
        ['LEFT', 'UP'], ['LEFT', 'DOWN'], ['RIGHT', 'UP'], ['RIGHT', 'DOWN'],
        ['UP', 'Z'], ['LEFT', 'UP', 'Z'], ['RIGHT', 'UP', 'Z'],
        ['UP', 'X'], ['LEFT', 'UP', 'X'], ['RIGHT', 'UP', 'X'],
        ['LEFT', 'X'], ['RIGHT', 'X'], ['LEFT', 'DOWN', 'X'], ['RIGHT', 'DOWN', 'X'],
        ['LEFT', 'A'], ['RIGHT', 'A'], ['LEFT', 'UP', 'A'], ['RIGHT', 'UP', 'A'], ['LEFT', 'DOWN', 'A'], ['RIGHT', 'DOWN', 'A'],
        ['DOWN', 'X'],
        ['DOWN', 'A'], 
        ['DOWN', 'C'], ['LEFT', 'DOWN', 'C'], ['RIGHT', 'DOWN', 'C'],
        ['DOWN', 'Y'], ['DOWN', 'LEFT', 'Y'], ['DOWN', 'RIGHT', 'Y'],
        ['LEFT', 'C', 'Z'], ['RIGHT', 'C', 'Z']]
        self._actions = []
        for action in actions:
            arr = np.array([False] * 12)
            for button in action:
                arr[buttons.index(button)] = True
            self._actions.append(arr)
        self.action_space = gym.spaces.Discrete(len(self._actions))

    def action(self, a): # pylint: disable=W0221
        return self._actions[a].copy()

Среда (игра)
*Не забудь добавить файлы!*

In [ ]:
gamename = 'MortalKombatII-Genesis'
#env.close()  #для изменеия
env = retro.make(gamename, 
                 state='FirstFightJax', 
                 #record = '/gdrive/My Drive/Retro/mkii/records/',  #для записи
                 obs_type=retro.Observations.IMAGE)
env = Discretizer(env)
env = DummyVecEnv([lambda: env])

# Модель

Создание модели

In [ ]:
modelname = 'subzero-ppo2'
model = PPO2(CnnPolicy, env, n_steps=1024, verbose=1)  #создание модели

Загрузка модели

In [ ]:
modelname = 'subzero-ppo2'
model = PPO2.load('/gdrive/My Drive/Retro/mkii/training_checkpoints/' + modelname +'_7400000_steps')
model.set_env(env)                                      #загрузка модели

Обучение модели

In [ ]:
modelname = 'subzero-ppo2'
checkpoint_callback = CheckpointCallback(save_freq=200000, 
                                         save_path='/gdrive/My Drive/Retro/mkii/new/training_checkpoints/', 
                                         name_prefix=modelname)
model.learn(total_timesteps=10000000, 
            callback=checkpoint_callback)    #обучение и сохранение модели
model.save("/gdrive/My Drive/Retro/mkii/" + modelname)

Выходные данные были обрезаны до нескольких последних строк (5000).
| policy_entropy     | 2.1351495     |
| policy_loss        | -0.0036616751 |
| serial_timesteps   | 7012352       |
| time_elapsed       | 3.37e+04      |
| total_timesteps    | 7012352       |
| value_loss         | 103942.086    |
--------------------------------------
-------------------------------------
| approxkl           | 0.017283052  |
| clipfrac           | 0.044189453  |
| explained_variance | 0.0284       |
| fps                | 209          |
| n_updates          | 6849         |
| policy_entropy     | 1.2670101    |
| policy_loss        | -0.012549325 |
| serial_timesteps   | 7013376      |
| time_elapsed       | 3.38e+04     |
| total_timesteps    | 7013376      |
| value_loss         | 104596.84    |
-------------------------------------
-------------------------------------
| approxkl           | 0.0080550155 |
| clipfrac           | 0.068359375  |
| explained_variance | 0.00994      |
| fps        

# Результаты

Видео и тестирование

In [ ]:
modelname = 'subzero-ppo2'
model = PPO2.load('/gdrive/My Drive/Retro/mkii/training_checkpoints/' + modelname +'_7400000_steps')
model.set_env(env)
mreward = -30000
num = 10000
sreward = 0
breward = 0
for i in range(num):
  obs = env.reset()
  done = False
  reward = 0
  while True:
    actions, _states = model.predict(obs)
    obs, rew, done, info = env.step(actions)
    reward += rew
    if done:
      break
  sreward = sreward + reward
  breward = breward + (reward + 10000)
  print(i, reward, sreward/(i+1), breward/(i+1))
  if reward > mreward:
    mreward = reward
    print ('!')

Loading a model without an environment, this model cannot be trained until it has a valid environment.
0 [10276.] [10276.] [20276.]
!
1 [-4956.] [2660.] [12660.]
2 [112.] [1810.6666] [11810.667]
3 [166.] [1399.5] [11399.5]
4 [5037.] [2127.] [12127.]
5 [30303.] [6823.] [16823.]
!
6 [5168.] [6586.5713] [16586.572]
7 [25265.] [8921.375] [18921.375]
8 [-10062.] [6812.1113] [16812.111]
9 [10207.] [7151.6] [17151.6]
10 [-10139.] [5579.727] [15579.728]
11 [-5028.] [4695.75] [14695.75]
12 [25182.] [6271.615] [16271.615]
13 [-5000.] [5466.5] [15466.5]
14 [20125.] [6443.7334] [16443.732]
15 [15199.] [6990.9375] [16990.938]
16 [15257.] [7477.1763] [17477.176]
17 [25201.] [8461.833] [18461.834]
18 [20096.] [9074.158] [19074.158]
19 [10121.] [9126.5] [19126.5]
20 [-10061.] [8212.81] [18212.809]
21 [5095.] [8071.091] [18071.092]
22 [30070.] [9027.565] [19027.564]
23 [-5071.] [8440.125] [18440.125]
24 [5127.] [8307.6] [18307.6]
25 [4944.] [8178.231] [18178.23]
26 [25184.] [8808.074] [18808.074]
27 [5

KeyboardInterrupt: ignored

Видео в mp4

In [ ]:
!python /usr/local/lib/python3.6/dist-packages/retro/scripts/playback_movie.py /gdrive/My\ Drive/Retro/mkii/records/MortalKombatII-Genesis-FirstFightJax-000052.bk2

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

# Доп?

In [ ]:
sts = ['jax',
       'rayden',
       'kitana',
       'kung_lao',
       'baraka',
       'reptile',
       'cage',
       'scorpion',
       'mileena',
       'subzero',
       'liu_kang',
       'jade',
       'shang_tsung',
       'kintaro',
       'shao_kahn']

gamename = 'MortalKombatII-Genesis'

modelname = 'subzero-ppo2'

#env.close()
env = retro.make(gamename, 
                 state='jax', 
                 #record = '/gdrive/My Drive/Retro/mkii/records/',
                 obs_type=retro.Observations.IMAGE)
env = Discretizer(env)
env = DummyVecEnv([lambda: env])

modelname = 'subzero-ppo2'
#model = PPO2(CnnPolicy, env, n_steps=1024, verbose=1)  #создание модели
model = PPO2.load('/gdrive/My Drive/Retro/mkii/training_checkpoints/' + modelname +'_7400000_steps')
model.set_env(env)   

model.learn(total_timesteps=100000)
model.save('/gdrive/My Drive/Retro/mkii/extra_checkpoints/' + modelname + '_first')
env.close()

for st in sts:
    print(st)
    env = retro.make(gamename, 
                 state=st, 
                 #record = '/gdrive/My Drive/Retro/mkii/records/',
                 obs_type=retro.Observations.IMAGE)
    env = Discretizer(env)
    env = DummyVecEnv([lambda: env])
    model.set_env(env)
    model.learn(total_timesteps=500000)
    model.save('/gdrive/My Drive/Retro/mkii/extra_checkpoints/' + modelname + '_' + st)
    env.close()

Выходные данные были обрезаны до нескольких последних строк (5000).
| policy_entropy     | 0.9837276     |
| policy_loss        | 0.00015749106 |
| serial_timesteps   | 29696         |
| time_elapsed       | 132           |
| total_timesteps    | 29696         |
| value_loss         | 26283.643     |
--------------------------------------
------------------------------------
| approxkl           | 0.051116064 |
| clipfrac           | 0.049804688 |
| explained_variance | 0.967       |
| fps                | 216         |
| n_updates          | 30          |
| policy_entropy     | 0.4691352   |
| policy_loss        | 0.006678975 |
| serial_timesteps   | 30720       |
| time_elapsed       | 137         |
| total_timesteps    | 30720       |
| value_loss         | 3454.6445   |
------------------------------------
-------------------------------------
| approxkl           | 0.10099911   |
| clipfrac           | 0.07080078   |
| explained_variance | 0.942        |
| fps                | 215

In [ ]:
st = 'shao_kahn'
gamename = 'MortalKombatII-Genesis'
modelname = 'subzero-ppo2'

#env.close()
env = retro.make(gamename, 
                 state = st, 
                 #record = '/gdrive/My Drive/Retro/mkii/records/',
                 obs_type=retro.Observations.IMAGE)
env = Discretizer(env)
env = DummyVecEnv([lambda: env])

modelname = 'subzero-ppo2'
#model = PPO2(CnnPolicy, env, n_steps=1024, verbose=1)  #создание модели
model = PPO2.load('/gdrive/My Drive/Retro/mkii/extra_checkpoints/' + modelname +'_kintaro')
model.set_env(env)   

model.learn(total_timesteps=500000)
model.save('/gdrive/My Drive/Retro/mkii/extra_checkpoints/' + modelname + '_' + st)
env.close()

Выходные данные были обрезаны до нескольких последних строк (5000).
| policy_entropy     | 1.3829219    |
| policy_loss        | 0.0009703551 |
| serial_timesteps   | 106496       |
| time_elapsed       | 426          |
| total_timesteps    | 106496       |
| value_loss         | 10330.748    |
-------------------------------------
--------------------------------------
| approxkl           | 0.0           |
| clipfrac           | 0.0           |
| explained_variance | 0.918         |
| fps                | 255           |
| n_updates          | 105           |
| policy_entropy     | 0.0           |
| policy_loss        | -3.579771e-08 |
| serial_timesteps   | 107520        |
| time_elapsed       | 430           |
| total_timesteps    | 107520        |
| value_loss         | 131299.38     |
--------------------------------------
------------------------------------
| approxkl           | 0.010723099 |
| clipfrac           | 0.056884766 |
| explained_variance | 0.955       |
| fps      